In [5]:
from metalib.metagomano import MetaGO
from datetime import datetime, timedelta
import pytz as pytz
import MetaTrader5 as mt5

In [6]:
end_time = datetime.now(pytz.utc) + timedelta(hours=3)
start_time = end_time - timedelta(days=30)

In [7]:
import yaml
import warnings

warnings.filterwarnings("ignore")

processes = []

with open("../config/prod/metago.yaml", "r") as f:
    config_data = yaml.safe_load(f)

name = "EURUSD_H1_WEEKLY"
entry = config_data[name]


In [8]:
strategy_type = entry.pop("strategy_type")
init_args = entry.copy()

# Convert timeframe string (e.g. "TIMEFRAME_M1") to actual mt5 constant
if isinstance(init_args.get("timeframe"), str):
    init_args["timeframe"] = eval(init_args["timeframe"])

# Convert null active_hours to None
if "active_hours" in init_args and init_args["active_hours"] == "None":
    init_args["active_hours"] = None

In [9]:
strategy_registry = {
    "metago": {
        "class": MetaGO,
        "init_keys": [
            "symbols",
            "timeframe",
            "tag",
            "active_hours",
            "size_position",
            "mean_rev_tf"
        ]
    }
}

In [10]:
strategy_registry

{'metago': {'class': metalib.metagomano.MetaGO,
  'init_keys': ['symbols',
   'timeframe',
   'tag',
   'active_hours',
   'size_position',
   'mean_rev_tf']}}

In [11]:
init_args["mean_rev_tf"] = "weekly"

In [12]:
strategy_info = strategy_registry.get(strategy_type)
if not strategy_info:
    raise ValueError(f"Unknown strategy type: {strategy_type}")

StrategyClass = strategy_info["class"]
expected_keys = strategy_info["init_keys"]

# Ensure required keys are present
for key in expected_keys:
    if key not in init_args:
        raise ValueError(f"Missing required argument: {key}")

instance = StrategyClass(**{k: init_args[k] for k in expected_keys})
instance.connect()
instance.fit()

No model to fit for MetaGO(mano) pelo!


In [13]:
instance.debug=True

In [14]:
instance.run(start_time, end_time)

Last time in the index: 2025-11-05 00:00:00
metago_weekly_eur::: Computed Trend indicators
metago_weekly_eur::: Computed True opens (weekly, monthly) 
metago_weekly_eur::: Computed True opens diffs (weekly, monthly) 
metago_weekly_eur::: Computed ATR
metago_weekly_eur::: Merged indicators

--- DEBUGGING INFORMATION ---
metago_weekly_eur::: Current Volatility: 0.0048
metago_weekly_eur::: Current Close Prices: [1.14839, 1.1479300000000001, 1.14798, 1.14807, 1.14812]
metago_weekly_eur::: True Open Monthly (last 3 values): [1.16092, 1.16092, 1.16092]
metago_weekly_eur::: Uptrend Indicator (last 3 values): [False, False, False]
metago_weekly_eur::: Downtrend Indicator (last 3 values): [True, True, True]
metago_weekly_eur::: ATR (last 3 values): [0.0012385714285713747, 0.0011878571428570897, 0.001101428571428523]

metago_weekly_eur::: Open positions for strategy: metago_weekly_eur: True

metago_weekly_eur::: Long Signal Components:
    - All of last 4 returns (except last) are > 0: False
   

True

In [15]:
instance.are_positions_with_tag_open()

True